# park_piper

This notebook is meant to work out the logic for a commandline Python tool that can take an ArcGIS Online point hosted feature layer and assigns it to the park that it is in.

## Libraries

In [1]:
from arcgis.gis import GIS
from arcgis import features
from arcgis import geometry
import pandas as pd
import geopandas as gpd

from copy import deepcopy
from getpass import getpass

## Variables

In [2]:
PORTAL = "https://skarplab.maps.arcgis.com"
USERNAME = "skarplab"
PIPER_ITEM_ID = "e26633d5fe834e248c0339d34a0f5de8"
PARK_ITEM_ID = "ac9b0c87805c45668d30ce80f7e3b53d"

## Helper Functions

In [3]:
def agol_to_gdf(fset):
    gdf = gpd.read_file(fset.to_geojson)
    gdf.crs = {'init': f'epsg:{fset.spatial_reference["latestWkid"]}'}
    gdf['geometry'] = gdf['geometry'].to_crs(epsg = 4326)
    return gdf

## Authenticate

In [6]:
gis = GIS(PORTAL, "skarplab", getpass())

 ·········


In [71]:
def park_piper(gis, piper_item, piper_feature_id_field, piper_update_field, park_item):
    
    # Parks
    parks_flayer = gis.content.get(park_item[0]).layers[park_item[1]]
    parks_fset = parks_flayer.query()
    parks_gdf = agol_to_gdf(parks_fset)

    # Piper features

    piper_flayer = gis.content.get(piper_item[0]).layers[piper_item[1]]
    piper_full_fset = piper_flayer.query()
    piper_no_park_fset = piper_flayer.query(where = f"{piper_update_field} IS NULL")
    piper_no_park_gdf = agol_to_gdf(piper_no_park_fset)
    
    piper_no_park_fset

#     piper_gdf[piper_update_field] = piper_gdf.apply(lambda x: parks_gdf.loc[parks_gdf['geometry'].contains(x['geometry'])][piper_update_field].iloc[0], axis = 1)

#     overlap_rows = pd.merge(left = piper_fset.sdf, right = points_gdf, how = 'inner', on = 'OBJECTID')

#     features_for_update = []
#     all_features = points_fset.features
#     all_features[0]

#     for id in overlap_rows['OBJECTID']:
#         original_feature = [f for f in all_features if f.attributes['OBJECTID'] == id][0]
#         feature_to_be_updated = deepcopy(original_feature)
#         print(str(original_feature))

#         matching_row = points_gdf.where(points_gdf["OBJECTID"] == id).dropna()
#         print(matching_row["ParkName"].values[0])
#         feature_to_be_updated.attributes["ParkName"] = matching_row["ParkName"].values[0]

#         features_for_update.append(feature_to_be_updated)
#         print(f'\n{str(feature_to_be_updated)}')
#         print(type(feature_to_be_updated))
#         print("\n###############################################################\n")

#     points_item.edit_features(updates = features_for_update)

In [72]:
park_piper(gis, (PIPER_ITEM_ID, 0), 'OBJECTID', 'ParkName', (PARK_ITEM_ID, 0))